# Report 02
### Daniel Bishop

License: Attribution 4.0 International (CC BY 4.0) 

In [1]:
from thinkbayes2 import Pmf

## The Chess Problem
### Copied from Quiz 1

> Two identical twins are members of my chess club, but they never show up on the same day; in fact, they strictly alternate the days they show up. I can't tell them apart except that one is a better player than the other: Avery beats me 60% of the time and I beat Blake 70% of the time. If I play one twin on Monday and win, and the other twin on Tuesday and lose, which twin did I play on which day?

Since the two twins always alternate their attendance, there are only 2 possible hypotheses for which one showed up on each day.
* Avery and then Blake  
or
* Blake and then Avery

In [2]:
class Chess(Pmf):
    def __init__(self, hypos):
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
        self.day = False;
        
    def Update(self, data):
        self.day = not self.day; # move to next day
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()
    
    '''
    the odds of the player winning or losing vs each opponent
    '''
    player_outcomes = {
        'blake': {
            'win': .7,
            'lose': .3,
            'alternate': { # quick hack to let each hypo access the likelihoods of the other
                'win': .4,
                'lose': .6
            }
         },
        'avery': {
            'win': .4,
            'lose': .6,
            'alternate': {
                'win': .7,
                'lose': .3
            }
        }
    }
    
    '''
    data an be either 'win' or 'lose',
    the likelihood of each outcome changes based on the 'day' where each new piece of data
    represents a new day.
    
    the hypothesis is the opponent that the player faced on Monday, which is the first data
    and is also an even day.
    '''
    def Likelihood(self, data, hypo):
        if (self.day):
            return self.player_outcomes[hypo][data]
        else:
            # choose the likelihood of the other opponent because they swap on the odd days
            return self.player_outcomes[hypo]["alternate"][data]
            

We can now use our model to predict which player we faced on Monday, and which we faced on Tuesday.

In [3]:
pmf = Chess(['blake', 'avery'])
dataset = ['win']
for data in dataset:
    pmf.Update(data)
for hypo, prob in pmf.Items():
    print(hypo, prob)

blake 0.6363636363636364
avery 0.36363636363636365


Here we can see that from the win on Monday, it is likely that we played against Blake, since we are more likely to win versus him.

In [4]:
pmf = Chess(['blake', 'avery'])
dataset = ['win', 'lose']
for data in dataset:
    pmf.Update(data)
for hypo, prob in pmf.Items():
    print(hypo, prob)

blake 0.7777777777777778
avery 0.22222222222222224


After losing on Tuesday, we are even more certain that we played against Blake on Monday, since that would mean that we played Avery on Tuesday, and Avery is much more likely to beat us than Blake.

In [5]:
pmf = Chess(['blake', 'avery'])
dataset = ['win', 'win']
for data in dataset:
    pmf.Update(data)
for hypo, prob in pmf.Items():
    print(hypo, prob)

blake 0.5000000000000001
avery 0.5


However, if we had instead won on Tuesday, we would once again be unsure of which twin we had played on either day.

## Kim Rhode

From the chapter 4 exercises of Think Bayes:  

> At the 2016 Summer Olympics in the Women's Skeet event, Kim Rhode faced Wei Meng in the bronze medal match. They each hit 15 of 25 skeets, sending the match into sudden death. In the first round, both hit 1 of 2 skeets. In the next two rounds, they each hit 2 skeets. Finally, in the fourth round, Rhode hit 2 and Wei hit 1, so Rhode won the bronze medal, making her the first Summer Olympian to win an individual medal at six consecutive summer games.
  
> But after all that shooting, what is the probability that Rhode is actually a better shooter than Wei? If the same match were held again, what is the probability that Rhode would win?

From the single match that we are given information on, it can be extracted that Kim has an overall accuracy of 64.5% and that Wei has an overall accuracy of 61.3%. Using these probabilities, we can predict the likelihood that Kim Rhode would win a rematch.

In [6]:
import random

class Match():
    def __init__(self):
        self.kim_score = 0;
        self.wei_score = 0;

    '''
    simulate standard match with overtime and return name of winner
    '''
    def run(self):
        self.kim_score = 0;
        self.wei_score = 0;
        
        for i in range(25):
            if (random.random() <= .645):
                self.kim_score += 1
            if (random.random() <= .613):
                self.wei_score += 1
                
        if (self.kim_score == self.wei_score):
            self.overtime()
            
        if (self.kim_score > self.wei_score):
            return 'kim'
        else: 
            return 'wei'
            
    def overtime(self):
        for i in range(2):
            if (random.random() <= .645):
                self.kim_score += 1
            if (random.random() <= .613):
                self.wei_score += 1
        
        if (self.kim_score == self.wei_score):
            self.overtime()
        
kim_wins = 0;
wei_wins = 0;
match = Match()
num_matches = 100000
for i in range(num_matches):
    winner = match.run()
    if (winner == 'kim'):
        kim_wins += 1
    else:
        wei_wins += 1
        
print("kim wins: " + str(kim_wins/num_matches))
print("wei wins: " + str(wei_wins/num_matches))

kim wins: 0.59521
wei wins: 0.40479


From repeated simulation, we can estimate that Kim Rhode would win a rematch about 60% of the time, based on both shooters' accuracy during the Olympic match.

## Original Problem

## What's for Lunch

Suppose your parents take turns making you lunch for school. Your mom packs you a pb&j for lunch 50% of the time, and other food 50% of the time. Your dad only packs you a pb&j 14% of the time though. How can you figure out your parents' schedule for making your lunch?

In [9]:
class Schedule(Pmf):
    def __init__(self, hypos):
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
        self.day = 0;
        
    def Update(self, data):
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()
        self.day += 1
        if (self.day >= 5):
            self.day = 0
        
    def Likelihood(self, data, hypo):
        if (hypo[self.day] == 'M'):
            prob_pb = .5
        else:
            prob_pb = .14
            
        if (data == 'pb&j'):
            return prob_pb
        else:
            return 1 - prob_pb

Now that we have set up the model to accomodate the specific pick and drop method, we can update it with the picking of 3 black socks followed by 2 white socks.

In [10]:
import itertools

pmf = Schedule([i for i in itertools.product(['M', 'D'], ['M', 'D'], ['M', 'D'], ['M', 'D'], ['M', 'D'])])
dataset = ["pb&j", "other", "other", "pb&j", "pb&j"]
for data in dataset:
    pmf.Update(data)
    
pmf.Print()

('D', 'D', 'D', 'D', 'D') 0.004185653475329125
('D', 'D', 'D', 'D', 'M') 0.014948762411889733
('D', 'D', 'D', 'M', 'D') 0.014948762411889737
('D', 'D', 'D', 'M', 'M') 0.05338843718532048
('D', 'D', 'M', 'D', 'D') 0.0024335194624006545
('D', 'D', 'M', 'D', 'M') 0.008691140937145193
('D', 'D', 'M', 'M', 'D') 0.008691140937145193
('D', 'D', 'M', 'M', 'M') 0.03103978906123283
('D', 'M', 'D', 'D', 'D') 0.0024335194624006545
('D', 'M', 'D', 'D', 'M') 0.008691140937145193
('D', 'M', 'D', 'M', 'D') 0.008691140937145193
('D', 'M', 'D', 'M', 'M') 0.03103978906123283
('D', 'M', 'M', 'D', 'D') 0.0014148368967445669
('D', 'M', 'M', 'D', 'M') 0.005052988916944881
('D', 'M', 'M', 'M', 'D') 0.005052988916944881
('D', 'M', 'M', 'M', 'M') 0.01804638898908886
('M', 'D', 'D', 'D', 'D') 0.014948762411889737
('M', 'D', 'D', 'D', 'M') 0.05338843718532048
('M', 'D', 'D', 'M', 'D') 0.05338843718532048
('M', 'D', 'D', 'M', 'M') 0.1906729899475731
('M', 'D', 'M', 'D', 'D') 0.008691140937145193
('M', 'D', 'M', 'D

Based on the week's worth of data, it seems like the most likely schedule is:
* Monday: Mom
* Tuesday: Dad
* Wednesday: Dad
* Thursday: Mom
* Friday: Mom

Let's see what happens after feeding another week's worth of data into the model.

In [12]:
dataset = ["other", "pb&j", "other", "other", "pb&j"]
for data in dataset:
    pmf.Update(data)
    
pmf.Print()

('D', 'D', 'D', 'D', 'D') 0.0018661215411995263
('D', 'D', 'D', 'D', 'M') 0.023802570678565383
('D', 'D', 'D', 'M', 'D') 0.0038748370872083186
('D', 'D', 'D', 'M', 'M') 0.0494239424388816
('D', 'D', 'M', 'D', 'D') 0.0006307874328013542
('D', 'D', 'M', 'D', 'M') 0.008045758071445844
('D', 'D', 'M', 'M', 'D') 0.0013097745697702539
('D', 'D', 'M', 'M', 'M') 0.01670630828788588
('D', 'M', 'D', 'D', 'D') 0.003874837087208319
('D', 'M', 'D', 'D', 'M') 0.049423942438881596
('D', 'M', 'D', 'M', 'D') 0.00804575807144584
('D', 'M', 'D', 'M', 'M') 0.10262446519701325
('D', 'M', 'M', 'D', 'D') 0.001309774569770254
('D', 'M', 'M', 'D', 'M') 0.016706308287885885
('D', 'M', 'M', 'M', 'D') 0.0027196315817488654
('D', 'M', 'M', 'M', 'M') 0.03468917833863348
('M', 'D', 'D', 'D', 'D') 0.003874837087208319
('M', 'D', 'D', 'D', 'M') 0.0494239424388816
('M', 'D', 'D', 'M', 'D') 0.008045758071445842
('M', 'D', 'D', 'M', 'M') 0.10262446519701325
('M', 'D', 'M', 'D', 'D') 0.0013097745697702532
('M', 'D', 'M', 

With this new data, the new most likely schedule is:
* Monday: Mom
* Tuesday: Mom
* Wednesday: Dad
* Thursday: Mom
* Friday: Mom

As more data arrives, the model will become more accurate, letting you guess which parent will be making your lunch, as long as they keep the same schedule between weeks.